# DSCI 525: Milestone 1

Group 7:
- Alex Truong Hai Yen
- Peter Yang
- Joshua JiaJie Lim
- Rahul Kuriyedath

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
#%load_ext rpy2.ipython
%load_ext memory_profiler

## Downloading the data

In [3]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [5]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 34.1 s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 17.8 s


# Combining data CSVs

In [3]:
import pandas as pd
files = glob.glob('figshareairline/*.csv')
if 'figshareairline\\combined_data.csv' in (files):
    files.remove('figshareairline\\combined_data.csv')
del files[-3]

In [4]:
%%time
%memit

import pandas as pd # we do not use use_cols trick because we are loading all columns in the file
df = pd.concat((pd.read_csv(file)
                .assign(model=re.findall(r'[^\\/]+(?=_daily_rainfall_NSW)', file)[0])
                for file in files)
              )
df.to_csv("figshareairline/combined_data.csv", index = False)

peak memory: 87.02 MiB, increment: 0.26 MiB
Wall time: 8min 21s


In [5]:
%%sh
du -sh figshareairline/combined_data.csv

5.7G	figshareairline/combined_data.csv


In [6]:
%%time
df = pd.read_csv("figshareairline/combined_data.csv")

Wall time: 1min 44s


In [7]:
print(df.shape)

(62467843, 7)


In [8]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


**Comparison in run times and memory usages on different machines**

Machine | Peter (Windows 16Gb RAM) |Alex (Windows 8Gb RAM) |Joshua (Windows 16Gb RAM) |Rahul (Windows 16Gb RAM) 
---|---|---|---|---
Download data - wall timne| 34.1s | 12min 38s
Unzip data - wall time| 17.8s| 25.5s 
Combine data - wall time| 6min 10s| 8 mins 21s
Combine data - peak memory | 110.30 MiB| 87.02 MiB
Read combined data - wall time | 53.6s | 1min 44s

**Comment:**

- Peter's machine can run the task well and faster than Alex's given the bigger RAM